In [7]:
import requests

PHI3_URL = "https://cu-vertical-dimensional-continuity.trycloudflare.com/phi3/generate"
PHI3_TOKEN = "AEulafWZQ1xhIkeLgjcp5nhL7d0cLPWiXAU34DTVvXI"  # Use the Token provided in the API docs: https://github.com/adventofmultimodalai/resources/blob/main/api.md#phi3-api-guide


def call_api(system, prompt, max_tokens, return_sentences_count=1, context=""):
    headers = {
        "Authorization": f"Bearer {PHI3_TOKEN}",
        "Content-Type": "application/json",
    }
    payload = {
        "inputs": f"<|system|>\n{system}<|end|>\n<|context|>{context}<|end|>\n<|user|>\n{prompt}<|end|>\n<|assistant|>",
        "parameters": {"max_new_tokens": max_tokens, "temperature": 0.1},  # Limit to a few words
    }
    response = requests.post(PHI3_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json().get("generated_text", "").split(". ")[:return_sentences_count]
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []


# print(call_api("answer the user's query. Caution: use strictly upto 100 tokens", "what is a dog?", 100, 2))

In [8]:
def generate_story_outline(prompt, max_tokens):
    headers = {
        "Authorization": f"Bearer {PHI3_TOKEN}",
        "Content-Type": "application/json",
    }
    payload = {
        "inputs": f"<|system|>\nYou are a creative manga story writer. Also breakdown the story into independent parts such that each can be represented by a picture. Prepend each picture description with `Picture`<|end|>\n<|user|>\n{prompt}<|end|>\n<|assistant|>",
        "parameters": {"max_new_tokens": max_tokens},
    }
    response = requests.post(PHI3_URL, headers=headers, json=payload)
    if response.status_code == 200:
        return response.json().get("generated_text", "").split("\n")
    else:
        print(f"Error: {response.status_code}, {response.text}")
        return []


class Manga:
    def __init__(self, story=None, style_keywords=[]):
        self.story = story
        self.style_keywords = style_keywords
        self.strip_shared_idx = 0
        self.strips = []
        self.strip_image_descriptions = []
        self.title = ""
        self.poster = ""
        if story is None:
            return
        self.strips = []
        self.title = story[0]
        for para in story:
            if para[:8] == "Picture:":
                self.strips.append(para)
        self.poster = call_api(
            "You are a creative director who is assigned the task to create posters",
            f"Create a poster for the following story, described in 3 sentences: {story}",
            40,
            3,
        )

    def __str__(self):
        rt = ""
        rt += "Title" + self.title + "\n"
        rt += "Poster details" + self.poster + "\n"
        for strip in self.strips:
            rt += strip + "\n"
        return rt

    def generate_strip_descriptions(self):
        for strip in self.strips:
            print("strip:", strip)
            called = call_api(
                "You are a manga artist. You know the best way to represent a scene or a story part as an image. Your output will be used to generate images. Therefore, you need to be very descriptive and clear. Your sentences must be self-contained and should not require any additional context.",
                f"Identify the following scene, represent in a single image-generation prompt strictly limited to 200 characters only: {strip}",
                40,
                3,
                context=self.story,
            )
            print("generated: ", called)
            self.strip_image_descriptions.append(called)

    def generate_poster(self):
        self.poster = "".join(
            call_api(
                "You are a creative artist who creates cinematic posters",
                f"Create a poster description for the following story (that can be used for image generation) and limit it strictly to 200 characters: {self.strips}",
                40,
                3,
            )
        )

    def generate_style_keywords(self, style_hint):
        self.style_keywords = call_api(
            "You are a manga artist. You know the best way to represent a scene or a story part as an image. Your output will be used to generate images. Therefore, you need to be very descriptive and clear. Your sentences must be self-contained and should not require any additional context.",
            f"Generate style keywords for the following style hint: {style_hint}",
            40,
            3,
            context=self.story,
        )

    def add_strip(self, strip_text):
        self.strips.append(strip_text)
        return

    def share_strip_with_idx(self, idx):
        return self.strip[idx]

    def share_strip(self):
        strip = self.strip[self.strip_shared_idx]
        self.strip_shared_idx = self.strip_shared_idx + 1
        return strip


# Example usage
prompt = """Create a manga adaptation of Sachin Tendulkar's autobiography, Playing It My Way. Focus on key moments from his life, including:
Early Years: Show young Sachin's passion for cricket, his first bat, and training with coach Ramakant Achrekar.
Career Highlights: Capture iconic matches, such as his debut against Pakistan and the famous Sharjah innings.
Personal Struggles: Illustrate the challenges he faced, including injuries and the loss of his father.
Legacy: Conclude with his emotional farewell match and reflections on his journey.
Use dynamic art to convey the excitement of cricket and emotional depth in personal moments. Highlight themes of dedication, family support, and resilience."""
story_outline = [
    "**Picture 1:**",
    "A young Sachin Tendulkar, with a determined look in his eyes, standing in a cricket field with a bat in his hands. The sun is setting, casting a warm glow over the scene. The bat is old and worn, symbolizing his early years and dedication to the sport.",
    "",
    "**Picture 2:**",
    "A close-up of Sachin's face, showing his excitement and focus as he practices with his coach, Ramakant Achrekar. The background is a cricket ground, with a crowd of young cricketers watching in awe. The coach's stern expression and Sachin's intense gaze convey the seriousness of their training sessions.",
    "",
    "**Picture 3:**",
    "A dramatic scene of Sachin's debut match against Pakistan. The stadium is packed, and the atmosphere is tense. Sachin is on the field, with a determined look on his face. The crowd is cheering, and the pressure is palpable. The focus is on Sachin's face, showing his determination and focus.",
    "",
    "**Picture 4:**",
    "A heartwarming scene of Sachin's family, including his mother and sister, watching him play in a match. The background is a cricket stadium, with Sachin in the center, looking focused and determined. The family's expressions show their pride and support for Sachin.",
    "",
    "**Picture 5:**",
    "A tense scene of Sachin's injury, with him lying on the ground, clutching his leg. The background is a cricket field, with a crowd of worried spectators. The focus is on Sachin's face, showing his pain and determination to recover.",
    "",
    "**Picture 6:**",
    "A poignant scene of Sachin's father's death, with Sachin sitting alone in a room, looking sad and lost. The background is a hospital room, with Sachin's father lying in a bed. The focus is on Sachin's face, showing his grief and loss.",
    "",
    "**Picture 7:**",
    "A dramatic scene of Sachin's farewell match, with him playing his final match. The stadium is packed, and the atmosphere is emotional. Sachin is on the field, with a determined look on his face. The focus is on Sachin's face, showing his emotions and the weight of his career.",
    "",
    "**Picture 8:**",
    "A reflective scene of Sachin, sitting alone in a room, looking at old cricket memorabilia. The background is a cricket room, with old trophies and photographs. The focus is on Sachin's face, showing his reflections on his journey and the legacy he has left behind.",
    "",
    "**Picture 9:**",
    "A heartwarming scene of Sachin's family, including his mother and sister, celebrating his retirement. The background is a cricket stadium, with Sachin's family and friends cheering. The focus is on Sachin's family, showing their pride and love for Sachin.",
    "",
    "**Picture 10:**",
    "A final scene of Sachin, standing on a podium, receiving a prestigious award. The background is a grand ceremony, with Sachin's family and friends cheering. The focus is on Sachin's face, showing his pride and gratitude for the recognition. The scene ends with Sachin looking out at the crowd, a symbol of his dedication, resilience, and legacy.",
]
if False:
    # regenerate only when needed to avoid long API
    story_outline = generate_story_outline(prompt, 2000)
print("Story Outline:", story_outline)
if not story_outline:
    print("Failed to generate story outline")
    exit()
the_manga = Manga(story_outline, ["manga", "noir", "detective", "supernatural", "dog"])
for text in story_outline:
    if text[:9] == "**Picture" or text[:7] == "Picture":
        the_manga.add_strip(text[8:])
the_manga.generate_poster()
# print(the_manga)
print(the_manga.poster)
the_manga.generate_strip_descriptions()
print("\nStrip Desciptions: ", the_manga.strip_image_descriptions)

Story Outline: ['**Picture 1:**', 'A young Sachin Tendulkar, with a determined look in his eyes, standing in a cricket field with a bat in his hands. The sun is setting, casting a warm glow over the scene. The bat is old and worn, symbolizing his early years and dedication to the sport.', '', '**Picture 2:**', "A close-up of Sachin's face, showing his excitement and focus as he practices with his coach, Ramakant Achrekar. The background is a cricket ground, with a crowd of young cricketers watching in awe. The coach's stern expression and Sachin's intense gaze convey the seriousness of their training sessions.", '', '**Picture 3:**', "A dramatic scene of Sachin's debut match against Pakistan. The stadium is packed, and the atmosphere is tense. Sachin is on the field, with a determined look on his face. The crowd is cheering, and the pressure is palpable. The focus is on Sachin's face, showing his determination and focus.", '', '**Picture 4:**', "A heartwarming scene of Sachin's family,

In [ ]:
FLUX_URL = "https://maintained-thai-filter-four.trycloudflare.com/imagine/generate"
FLUX_TOKEN = "calm-bold-tree-bd1b1c8bc141403231e7f0a8"  # Check the API docs for details.

def generate_image(description, output_file):
    headers = {
        "Authorization": f"Bearer {FLUX_TOKEN}",
        "Content-Type": "application/json"
    }
    payload = {
        "prompt": description[:200],  # Max 200 characters for prompt
        "img_size": 512,
        "guidance_scale": 7.5,
        "num_inference_steps": 50
    }
    response = requests.post(FLUX_URL, headers=headers, json=payload)
    if response.status_code == 200:
        with open(output_file, "wb") as f:
            f.write(response.content)
        print(f"Image saved to {output_file}")
    else:
        print(f"Error: {response.status_code}, {response.text}")

# Example usage
generate_image(f"[{str(the_manga.style_keywords)}] Add TEXT in image{the_manga.title}. Image description: {the_manga.poster}", "poster.png")
for idx, strip in enumerate(the_manga.strip_image_descriptions):
    generate_image(f"[{str(the_manga.style_keywords)}] {str(strip)}", f"scene_{idx}.png")

Image saved to poster.png
Image saved to scene_0.png
Image saved to scene_1.png
Image saved to scene_2.png
Image saved to scene_3.png
Image saved to scene_4.png
Image saved to scene_5.png
Image saved to scene_6.png
Image saved to scene_7.png
Image saved to scene_8.png
Image saved to scene_9.png
Image saved to scene_10.png
Image saved to scene_11.png


In [6]:
generate_image(f"Add TEXT in image{the_manga.title}. Image description: {the_manga.poster}", "poster.png")


Image saved to poster.png


In [ ]:
print(story_outline)

['**Picture 1:**', 'A young Sachin Tendulkar, with a determined look in his eyes, standing in a cricket field with a bat in his hands. The sun is setting, casting a warm glow over the scene. The bat is old and worn, symbolizing his early years and dedication to the sport.', '', '**Picture 2:**', 'Sachin, a teenager, practicing under the guidance of Coach Ramakant Ach']
